In [1]:
!pip install pyTelegramBotAPI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.1/243.1 kB 683.6 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.14.0-py3-none-any.whl size=215251 sha256=510df884c54d765c329f0989f46f317c5b4f653457730c179bebbacfa90aef8f
  Stored in directory: /Users/user/Library/Caches/pip/wheels/f0/ae/f9/dfb61e8eab750a5cffaba33a8d1570c6ad0b8bacfd029d3261
Successfully built pyTelegramBotAPI

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


Бот-книга рецептов:</br>
•  Кнопки: "Рецепт дня", "Вегетарианский рецепт", "Поиск рецепта".</br>
•  Функции: Предоставляет различные рецепты.</br>

In [ ]:
import telebot
from telebot import types
import requests
import math

import datetime

bot = telebot.TeleBot(token)


@bot.message_handler(commands=['start'])
def send_keyboard(call, text="Привет, чем я могу тебе помочь?"):
    keyboard = types.ReplyKeyboardMarkup(row_width=2)
    itembtn1 = types.KeyboardButton('Погодку покажи')
    keyboard.add(itembtn1)

    msg = bot.send_message(
        call.chat.id,
        text=text, 
        reply_markup=keyboard
    )

    bot.register_next_step_handler(msg, callback_worker)


@bot.message_handler(content_types=['text'])
def handle_docs_audio(message):
    send_keyboard(message, text="Я не понимаю :-( Выберите один из пунктов меню:")


def callback_worker(call):
    if call.text == "Погодку покажи":
        msg = bot.send_message(call.chat.id,
                               text='Сейчас такая погода!')
        get_wheather(msg)


bot.polling(none_stop=True)